## 가변 길이 인수

In [2]:
import numpy as np

class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

  def backward(self):
    if self.grad is None:
      self.grad=np.ones_like(self.data)
    funcs=[self.creator]
    while funcs:
      f=funcs.pop() # 함수 가져옴
      x,y= f.input, f.output # 함수 입출력 가져옴
      x.grad=f.backward(y.grad) #self.grad 하면 안됨(1.0만 호출) y.grad해야 실제 f.output계속 호출

      if x.creator is not None:
        funcs.append(x.creator) # 하나 앞 변수의 함수를 리스트에 추가

In [3]:
def as_array(x):
  if np.isscalar(x): #numpy.float64같은 스칼라타입인지 확인
    return np.array(x)
  return x

class Function:
  def __call__(self, inputs):
    xs=[x.data for x in inputs] # inputs: Variable 변수들이 들어있는 리스트
    ys=self.forward(xs)
    outputs=[Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs=inputs
    self.outputs=outputs
    return outputs

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys): # gy는 출력쪽에서 전해지는 미분값
    raise NotImplementedError()

In [4]:
class Add(Function):
  def forward(self, xs): # 인수,반환값==리스트 혹은 튜플
    x0,x1=xs
    y=x0+x1
    return (y,)

xs=[Variable(np.array(2)),Variable(np.array(3))]
f=Add()
ys=f(xs)
y=ys[0]
print(y.data)

5


In [5]:
# 입력변수를 리스트에 담아서 건네거나, 반환값인 튜플을 풀어야 하는 것은 번거로움
# 가변길이인수(argument)로 개선
class Function:
  def __call__(self, *inputs): # 수정: *은 임의 개수의 인수 나타낸다
    xs=[x.data for x in inputs] # inputs: Variable 변수들이 들어있는 리스트
    ys=self.forward(xs)
    if not isinstance(ys, tuple):
      ys=(ys,)
    outputs=[Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs=inputs
    self.outputs=outputs
    return outputs if len(outputs)>1 else outputs[0] # 수정

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

class Add(Function):
  def forward(self, xs): # 인수,반환값==리스트 혹은 튜플
    x0,x1=xs
    y=x0+x1
    return (y,)

In [6]:
x0=Variable(np.array(2))
x1=Variable(np.array(3))
f=Add()
y=f(x0,x1)
print(y.data)

5


## 가변 길이 인수 - 역전파편


In [7]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

  def backward(self):
    if self.grad is None:
      self.grad=np.ones_like(self.data)
    funcs=[self.creator]
    while funcs:
      f=funcs.pop() # 함수 가져옴
      gys=[output.grad for output in f.outputs]
      gxs=f.backward(*gys) # 리스트 언팩킹 f.backward(np.array(1.0), np.array(2.0))
      if not isinstance(gxs,tuple):
        gxs=(gxs,)
      for x, gx in zip(f.inputs, gxs):
        x.grad=gx

        if x.creator is not None:
          funcs.append(x.creator)

class Function:
  def __call__(self, *inputs): # 수정: *은 임의 개수의 인수 나타낸다
    xs=[x.data for x in inputs] # inputs: Variable 변수들이 들어있는 리스트
    ys=self.forward(*xs) #개별 인수 받게 구현되어있으니 언팩킹
    if not isinstance(ys, tuple):
      ys=(ys,)
    outputs=[Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs=inputs
    self.outputs=outputs
    return outputs if len(outputs)>1 else outputs[0] # 수정

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

# 가변길이지원
class Square(Function):
  def forward(self, x):
    y=x**2
    return y
  def backward(self, gy):
    x=self.inputs[0].data # 수정전) self.input.data
    gx=2*x*gy
    return gx

class Add(Function):
  def forward(self, x0,x1):
    y=x0+x1
    return y
  def backward(self, gy):
    return gy,gy #상류에서 흘러오는 미분값을 그대로 흘려보내는 것이 덧셈의 역전파(미분시, gy*1이라서)

def square(x):
  return Square()(x)

def add(x0,x1):
  return Add()(x0,x1)

In [8]:
x=Variable(np.array(2.0))
y=Variable(np.array(3.0))

z=add(square(x),square(y))
z.backward()
print("z.data:",z.data)
print(x.grad)
print(y.grad)
print(z.grad)

z.data: 13.0
4.0
6.0
1.0


## 같은 변수 반복 사용시의 문제


In [10]:
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad) # 답은 2

1.0


In [11]:
# y=add(x,x) <=미분결과 틀림
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

  def backward(self):
    if self.grad is None:
      self.grad=np.ones_like(self.data)
    funcs=[self.creator]
    while funcs:
      f=funcs.pop() # 함수 가져옴
      gys=[output.grad for output in f.outputs]
      gxs=f.backward(*gys) # 리스트 언팩킹 f.backward(np.array(1.0), np.array(2.0))
      if not isinstance(gxs,tuple):
        gxs=(gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad=gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          funcs.append(x.creator)

In [12]:
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad)

2.0


In [13]:
x=Variable(np.array(3.0))
y=add(add(x,x),x)
y.backward()
print(x.grad)

3.0


In [14]:
# 같은 변수 사용하여 다른 계산 하면 문제 발생
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad)
y=add(add(x,x),x)
y.backward()
print(x.grad)

2.0
5.0


In [15]:
# 미분값 초기화하는 cleargrad메서드 추가
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

  def backward(self):
    if self.grad is None:
      self.grad=np.ones_like(self.data)
    funcs=[self.creator]
    while funcs:
      f=funcs.pop()
      gys=[output.grad for output in f.outputs]
      gxs=f.backward(*gys)
      if not isinstance(gxs,tuple):
        gxs=(gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad=gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          funcs.append(x.creator)

  def cleargrad(self): # 추가
    self.grad=None

In [16]:
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad)

x.cleargrad() # 추가

y=add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0


## 복잡한 계산 그래프 (함수와 변수의 세대)

In [18]:
# 역전파시 최근 세대의 함수부터 꺼내지 않는다면 로직 오류
# 순전파시 세대 부여하는 코드 추가
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.grad=None
    self.creator=None
    self.generation=0 # 추가

  def set_creator(self, func):
    self.creator=func
    self.generation=func.generation+1 # 부모 세대 함수보다 1만큼 큰 값 설정

  def backward(self):
    if self.grad is None:
      self.grad=np.ones_like(self.data)

    # 수정-funcs정렬
    funcs=[]
    seen_set=set()
    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x:x.generation)

    add_func(self.creator)

    while funcs:
      f=funcs.pop()
      gys=[output.grad for output in f.outputs]
      gxs=f.backward(*gys)
      if not isinstance(gxs,tuple):
        gxs=(gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad=gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator) # 수정

  def cleargrad(self):
    self.grad=None

class Function:
  def __call__(self, *inputs):
    xs=[x.data for x in inputs]
    ys=self.forward(*xs)
    if not isinstance(ys, tuple):
      ys=(ys,)
    outputs=[Variable(as_array(y)) for y in ys]

    # 입력 변수 세대중에 가장 큰 세대 값으로 함수의 세대 설정
    self.generation=max([x.generation for x in inputs])
    for output in outputs:
      output.set_creator(self)
    self.inputs=inputs
    self.outputs=outputs
    return outputs if len(outputs)>1 else outputs[0] # 수정

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

# 가변길이지원
class Square(Function):
  def forward(self, x):
    y=x**2
    return y
  def backward(self, gy):
    x=self.inputs[0].data # 수정전) self.input.data
    gx=2*x*gy
    return gx

class Add(Function):
  def forward(self, x0,x1):
    y=x0+x1
    return y
  def backward(self, gy):
    return gy,gy #상류에서 흘러오는 미분값을 그대로 흘려보내는 것이 덧셈의 역전파(미분시, gy*1이라서)

def square(x):
  return Square()(x)

def add(x0,x1):
  return Add()(x0,x1)

In [20]:
# 2*x^4 미분 문제
x=Variable(np.array(2.0))
a=square(x)
y=add(square(a),square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


## 메모리 관리와 순환 참조 (성능향상)

In [29]:
'''
파이썬은 필요 없어진 객체를 메모리에서 자동 삭제하지만 코드를 제대로 작성하지 않으면 메모리 누수 문제 발생 (특히 거대 신경망이면)
필요 없어졌다의 기준
1) 참조 수 카운트 (참조 카운트/기본)
-대입 연산자 사용
-함수에 인수로 전달
-컨테이너 타입 객체에 추가(리스트 튜플 등)
2) 세대 기준으로 객체 회수 (GC)
-순환 참조를 처리
'''
# weakref 모듈: 약한 참조(다른 객체 참조하되, 참조 카운트는 증가x)
import weakref
class Variable:
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.name=name # 많은 변수 처리 -> 이름 필요 -> 인스턴스 변수 추가
    self.grad=None
    self.creator=None
    self.generation=0

  def set_creator(self, func):
    self.creator=func
    self.generation=func.generation+1 # 부모 세대 함수보다 1만큼 큰 값 설정

  def backward(self, retain_grad=False): # retain_grad: 필요 없는 미분값 삭제 / 보통 말단 변수 미분값만이 필요
    if self.grad is None:
      self.grad=np.ones_like(self.data)

    funcs=[]
    seen_set=set()
    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x:x.generation)

    add_func(self.creator)

    while funcs:
      f=funcs.pop()
      gys=[output().grad for output in f.outputs] #약한 참조로 바꿨으므로 ()추가하여 수정
      gxs=f.backward(*gys)
      if not isinstance(gxs,tuple):
        gxs=(gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad=gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator) # 수정

      if not retain_grad: # 말단 변수 아닐시 미분값 삭제
        for y in f.outputs:
          y().grad=None

  def cleargrad(self):
    self.grad=None

class Config: # 역전파 활성화 모드
  enable_backprop=True

class Function:
  def __call__(self, *inputs):
    xs=[x.data for x in inputs]
    ys=self.forward(*xs)
    if not isinstance(ys, tuple):
      ys=(ys,)
    outputs=[Variable(as_array(y)) for y in ys]

    if Config.enable_backprop: # 역전파 모드에서만 연결 연산 수행
      # 입력 변수 세대중에 가장 큰 세대 값으로 함수의 세대 설정
      self.generation=max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs=inputs
      self.outputs=[weakref.ref(output) for output in outputs] # 약한 참조로 수정

    return outputs if len(outputs)>1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

# 가변길이지원
class Square(Function):
  def forward(self, x):
    y=x**2
    return y
  def backward(self, gy):
    x=self.inputs[0].data
    gx=2*x*gy
    return gx

class Add(Function):
  def forward(self, x0,x1):
    y=x0+x1
    return y
  def backward(self, gy):
    return gy,gy #상류에서 흘러오는 미분값을 그대로 흘려보내는 것이 덧셈의 역전파(미분시, gy*1이라서)

def square(x):
  return Square()(x)

def add(x0,x1):
  return Add()(x0,x1)

In [30]:
for i in range(10):
  x=Variable(np.random.randn(10000)) # 거대데이터
  y=square(square(square(x)))
# for문 두번쨰 참조될 때 x,y덮어써짐, 이전 계산 그래프 참조 안하므로 자동 삭제됨(약한참조로 순환참조문제 해결)

In [31]:
Config.enable_backprop=True
x=Variable(np.ones((100,100,100)))
y=square(square(square(x)))
y.backward()

Config.enable_backprop=False
x=Variable(np.ones((100,100,100)))
y=square(square(square(x)))
Config.enable_backprop=True

In [32]:
# with문 사용한 모드 전환(후처리 자동 수행)
# with open('sample.txt','w') as f:
#   f.write("hello")
# # 파일 자동으로 닫음
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value=getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield # 컨텍스트 블록 실행 (이 부분에서 실행 흐름이 사용자 코드로 넘어감)
  finally:
    setattr(Config, name, old_value)

with using_config('enable_backprop', False):
  x=Variable(np.array(2.0))
  y=square(x)
# with문 바깥에서는 다시 True가 됨

# 코드가 길어지니 귀찮음->함수로
def no_grad():
  return using_config('enable_backprop',False)

with no_grad(): # 단순 순전파만 필요할때 연산 줄임
  x=Variable(np.array(2.0))
  y=square(x)

In [36]:
class Variable:
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data)

(100, 100, 100)

## 변수(Variable) 사용성 개선

In [62]:
import weakref
class Variable:
  __array_priority__=200
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.name=name # 많은 변수 처리 -> 이름 필요 -> 인스턴스 변수 추가
    self.grad=None
    self.creator=None
    self.generation=0

  def set_creator(self, func):
    self.creator=func
    self.generation=func.generation+1 # 부모 세대 함수보다 1만큼 큰 값 설정

  def backward(self, retain_grad=False): # retain_grad: 필요 없는 미분값 삭제 / 보통 말단 변수 미분값만이 필요
    if self.grad is None:
      self.grad=np.ones_like(self.data)

    funcs=[]
    seen_set=set()
    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x:x.generation)

    add_func(self.creator)

    while funcs:
      f=funcs.pop()
      gys=[output().grad for output in f.outputs] #약한 참조로 바꿨으므로 ()추가하여 수정
      gxs=f.backward(*gys)
      if not isinstance(gxs,tuple):
        gxs=(gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad=gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator) # 수정

      if not retain_grad: # 말단 변수 아닐시 미분값 삭제
        for y in f.outputs:
          y().grad=None

  def cleargrad(self):
    self.grad=None

  # 목표: Variable을 ndarray처럼 보이게 만드는 것
  @property # 인스턴스 변수처럼 사용할수 있게 함 x.shape() -> x.shape
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self): # 특수 메서드
    return len(self.data)

  def __repr__(self): # print함수 출력값 수정하려면  __repr__재정의
    if self.data is None:
      return 'variable(None)'
    p=str(self.data).replace('\n','\n'+''*9)
    return 'variable('+p+')'
  # 함수식 거추장-> 연산자 오버로드
  def __mul__(self, other): #self=a전달, other=b전달
    return mul(self, other)

# 또는
Variable.__add__=add # 함수 자체 할당

In [47]:
x=Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape, "and" ,len(x))

(2, 3) and 2


In [58]:
class Mul(Function):
  def forward(self, x0, x1):
    y=x0*x1
    return y

  def backward(self, gy):
    x0,x1=self.inputs[0].data, self.inputs[1].data
    return gy*x1, gy*x0
def mul(x0,x1):
  x1=as_array(x1) #x1가 float등일 경우 ndarray인스턴스로 변환
  return Mul()(x0,x1)

a=Variable(np.array(3.0))
b=Variable(np.array(2.0))
c=Variable(np.array(1.0))

y=add(mul(a,b),c)

y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [50]:
a=Variable(np.array(3.0))
b=Variable(np.array(2.0))
c=Variable(np.array(1.0))

y=a*b+c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [52]:
def as_variable(obj): # np인스턴스와 함꼐 입력되도 오류없이 수행
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)

class Function:
  def __call__(self, *inputs):
    inputs=[as_variable(x) for x in inputs]
    xs=[x.data for x in inputs]
    ys=self.forward(*xs)
    if not isinstance(ys, tuple):
      ys=(ys,)
    outputs=[Variable(as_array(y)) for y in ys]

    if Config.enable_backprop: # 역전파 모드에서만 연결 연산 수행
      # 입력 변수 세대중에 가장 큰 세대 값으로 함수의 세대 설정
      self.generation=max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs=inputs
      self.outputs=[weakref.ref(output) for output in outputs] # 약한 참조로 수정

    return outputs if len(outputs)>1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

class Square(Function):
  def forward(self, x):
    y=x**2
    return y
  def backward(self, gy):
    x=self.inputs[0].data
    gx=2*x*gy
    return gx

class Add(Function):
  def forward(self, x0,x1):
    y=x0+x1
    return y
  def backward(self, gy):
    return gy,gy #상류에서 흘러오는 미분값을 그대로 흘려보내는 것이 덧셈의 역전파(미분시, gy*1이라서)

def square(x):
  return Square()(x)

def add(x0,x1):
  return Add()(x0,x1)

In [53]:
x=Variable(np.array(2.0))
y=x+np.array(3.0)
print(y)

variable(5.0)


In [55]:
# 파이썬의 일반 float, int등과 함께 사용하기 (좌항이 float, int인 경우)
def add(x0,x1):
  x1=as_array(x1) #x1가 float등일 경우 ndarray인스턴스로 변환
  return Add()(x0,x1) # Function 내에서 Variable 인스턴스로 변환

In [57]:
x=Variable(np.array(2.0))
y=x+3.0
print(y)

variable(5.0)


In [59]:
# 연산자 오버로드 방향 문제 해결 (덧셈이나 곱셈이나 어차피 교환법칙 성립)
Variable.__add__=add
Variable.__radd__=add
Variable.__mul__=mul
Variable.__rmul__=mul


In [61]:
y=2.0*x+1.0
y

variable(5.0)

In [65]:
# 좌항이 ndarray인 경우 - 좌항인 ndarray인스턴스의 __add__메서드가 호출된다.
# 연산자 우선순위 추가 (Variable에서)
class Neg(Function):
  def forward(self,x):
    return -x
  def backward(self, gy):
    return -gy
def neg(x):
  return Neg()(x)
Variable.__neg__=neg

class Sub(Function):
  def forward(self, x0,x1):
    return x0-x1
  def backward(self, gy):
    return gy,-gy

def sub(x0,x1):
  x1=as_array(x1)
  return Sub()(x0,x1)
Variable.__sub__=sub

def rsub(x0,x1):
  x1=as_array(x1)
  return Sub()(x1,x0)
Variable.__rsub__=rsub

In [66]:
x=Variable(np.array(2.0))
y1=2.0-x
y2=x-1.0
print(y1)
print(y2)

variable(0.0)
variable(1.0)


In [68]:
class Div(Function):
  def forward(self, x0,x1):
    y=x0/x1
    return y
  def backward(self, gy):
    x0,x1=self.inputs[0].data, self.inputs[1].data
    return gy*(1/x1), gy*(-x0/x1**2)

def div(x0,x1):
  x1=as_array(x1)
  return Div()(x0,x1)
Variable.__truediv__=div
def rdiv(x0,x1):
  x1=as_array(x1)
  return Div()(x1,x0)
Variable.__rtruediv__=rdiv



*   모듈: 파이썬 파일(.py)
*   패키지: 여러 모듈을 묶은 것
*   라이브러리: 여러 패키지를 묶은 것



